# Q&A against Tabular Data from a CSV file  (experimental)

To really have a Smart Search Engine or Virtual assistant that can answer any question about your corporate documents, this "engine" must understand tabular data, aka, sources with tables, rows and columns with numbers. 
This is a different problem that simply looking for the top most similar results.  The concept of indexing, bringing top results, embedding, doing a cosine semantic search and summarize an answer, doesn't really apply to this problem.
We are dealing now with sources with Tables in which each row and column are related to each other, and in order to answer a question, all of the data is needed, not just top results.

In this notebook, the goal is to show how to deal with this kind of use cases. To continue with our Covid-19 theme, we will be using an open dataset called ["Covid Tracking Project"](https://covidtracking.com/data/download). The COVID Tracking Project dataset is a  CSV file that provides the latest numbers on tests, confirmed cases, hospitalizations, and patient outcomes from every US state and territory (they stopped tracking on March 7 2021).

Imagine that many documents on a data lake are tabular data, or that your use case is to ask questions in natural language to a LLM model and this model needs to get the context from a CSV file or even a SQL Database in order to answer the question. A GPT Smart Search Engine, must understand how to deal with this sources, understand the data and answer acoordingly.

In [4]:
import os
import pandas as pd
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

from app.prompts import CSV_PROMPT_PREFIX, CSV_PROMPT_SUFFIX

# Don't mess with this unless you really know what you are doing
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"

# Change these below with your own services credentials
AZURE_OPENAI_ENDPOINT = "https://openainlp1.openai.azure.com/"
AZURE_OPENAI_API_KEY = "9e64f10580c44989b8fabe9b23336344"

In [5]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"

## Download the dataset and load it into Pandas Dataframe

In [6]:
os.makedirs("data",exist_ok=True)

In [7]:
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/

--2023-05-17 15:24:27--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 104.21.14.119, 172.67.133.195, 2606:4700:3036::ac43:85c3, ...
Connecting to covidtracking.com (covidtracking.com)|104.21.14.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv.2’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.06s   

2023-05-17 15:24:28 (45.6 MB/s) - ‘./data/all-states-history.csv.2’ saved [2738601]



In [8]:
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (20780, 41)


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110


In [9]:
df.columns

Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease'

## Load our LLM and create our MRKL Agent

The implementation of Agents is inspired by two papers: the [MRKL Systems](https://arxiv.org/abs/2205.00445) paper (pronounced ‘miracle’ 😉) and the [ReAct](https://arxiv.org/abs/2210.03629) paper.

Agents are a way to leverage the ability of LLMs to understand and act on prompts. In essence, an Agent is an LLM that has been given a very clever initial prompt. The prompt tells the LLM to break down the process of answering a complex query into a sequence of steps that are resolved one at a time.

Agents become really cool when we combine them with ‘experts’, introduced in the MRKL paper. Simple example: an Agent might not have the inherent capability to reliably perform mathematical calculations by itself. However, we can introduce an expert - in this case a calculator, an expert at mathematical calculations. Now, when we need to perform a calculation, the Agent can call in the expert rather than trying to predict the result itself. This is actually the concept behind [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins).

In our case, in order to solve the problem "How do I ask questions to a tabular CSV file", we need this REACT/MRKL approach, in which we need to instruct the LLM that it needs to use an 'expert/tool' in order to read/load/understand/interact with a CSV tabular file.

OpenAI opened the world to a whole new concept. Libraries are being created fast and furious. We will be using [LangChain](https://docs.langchain.com/docs/) as our library to solve this problem, however there are others that we recommend: [HayStack](https://haystack.deepset.ai/) and [Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/whatissk).

In [10]:
# Let's ask a complex question that requires multiple steps to solve it, and where it is not clear.
# looking at the dataframea above, even by a human on what columns to use
QUESTION = 'How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states?'

In [11]:
# First we load our LLM: GPT-4 (you are welcome to try GPT-3.5-Turbo. You will see that GPT-3.5 
# does not have the cognitive capabilities to solve a complex question)
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0, max_tokens=500)

Now we need our agent and our expert/tool.  
LangChain has created an out-of-the-box agents that we can use to solve our Q&A to CSV tabular data file problem. For more informatio about tje **CSV Agent** click [HERE](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/csv.html)

In [12]:
agent = create_csv_agent(llm, file_url, verbose=True)

## Enjoy the response and the power of GPT-4 + REACT/MKRL approach

In [13]:
# We are doing a for loop to retry N times. This is because: 
# 1) GPT-4 is still in preview and the API is being very throttled and 
# 2) Because the LLM not always gives the answer on the exact format the agent needs and hence cannot be parsed

for i in range(5):
    try:
        response = agent.run(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX) 
        break
    except:
        response = "Error too many failed retries"
        continue
        
print(response)



> Entering new AgentExecutor chain...
Thought: 
Let's start by setting the pandas display options to show all the columns and getting the column names. Then we can use pandasql to query the dataframe and get the answer to the question.
Action: python_repl_ast
Action Input:
```
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv('covid_data.csv')
print(df.columns)
```
Observation: FileNotFoundError: [Errno 2] No such file or directory: 'covid_data.csv'
Thought:Oops, I need to make sure the file path is correct. Let me check where the file is located.
Action: python_repl_ast
Action Input:
```
import os
print(os.getcwd())
```/workspaces/GPT-Azure-Search-Engine

Observation: None
Thought:The current working directory is not printed. I need to change the directory to where the file is located.
Action: python_repl_ast
Action Input:
```
import os
os.chdir('/path/to/file')
print(os.getcwd())
```
Observation: FileNotFoundError: [Errno 2] No such file or directory: 

## Evaluation
Let's see if the answer is correct

In [14]:
#df['date'] = pd.to_datetime(df['date'])
july_2020 = df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]
texas_hospitalized_july_2020 = july_2020[july_2020['state'] == 'TX']['hospitalizedIncrease'].sum()
nationwide_hospitalized_july_2020 = july_2020['hospitalizedIncrease'].sum()

In [12]:
print( "TX:",texas_hospitalized_july_2020,"Nationwide:",nationwide_hospitalized_july_2020)

TX: 0 Nationwide: 63105


Bad pipe message: %s [b"\xaay\x1d\x9a\xef[:\x86\xc9\x0bq\x84r\xef\xdfSu\xf0 \xf0\x8b\x98\x18\xf1Z_\xff\x9aD\x94\xc5\xe2\xeeO\xdd\xe0\tP\xfa\xa4f9\x9ce\x9d\xe0\x0f1\xf8\xb8'\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00"]
Bad pipe message: %s [b'\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08']
Bad pipe message: %s [b'\x08\n\x08\x0b\x08\x04\x08', b'\x06\x04\x01\x05']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xde\x9dj\x03\xde6\x80\xe1v/\x15\xda6\xe7\xf5\x0e\xb5\xceBd\xc4\x1f']
Bad pipe message: %s [b'\x1d\x13\xc5\x8f>f\xde\x9c$${\xc3b\x85\xd1', b'\xe9 \x9bU"\xf2\xa6\x9bv$\xebiD\xa9\x1a~W\xcf\xbbi\x02\x8c\xce\x8b]\xc0 \xb0\xf5qM\xe0\x88\x1e\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f

It is Correct!

**Note**: Obviously, there were hospitalizations in Texas in July 2020 (Try asking ChatGPT), but this particular File, for some reason has 0 on the column "hospitalizedIncrease" for Texas in July 2020. This proves though that the model is NOT making up information or using prior knowledge, but instead using only the results of its calculation on this CSV file. That's what we need!

**Note 2**: You will also notice that if you run the above cell multiple times, not always you will get the same result. Sometimes it will even fail an error out. Why? 
1) This is still a very new field and LLMs and libraries still has a lot room to grow
2) Because for complex questions that require multiple steps to solve it, even humans make mistakes
3) Because if the column names are not clear, or ambiguous, or the data is not clean, it will make mistakes, just as humans would.

# Summary

So, we just solved our problem on how to ask questions in natural language to our Tabular data hosted on a CSV File.
With this approach you can see then that it is NOT necessary to make a dump of a database data into a CSV file and index that on a Search Engine, you don't even need to use the above approach and deal with a CSV data dump file. With the Agents framework, the best engineering decision is to interact directly with the data source API without the need to replicate the data in order to ask questions to it. Remember, GPT-4 can do SQL very well. 

Just think about this: if GPT-4 can do the above, imagine what GPT-5/6/7/8 will be able to do.

This is implemented in the App of this repo as well, where you will be able to upload any tabular CSV you like and ask questions to it

# Reference

- https://haystack.deepset.ai/blog/introducing-haystack-agents
- https://python.langchain.com/en/latest/modules/agents/agents.html
- https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/
- https://medium.com/@meghanheintz/intro-to-langchain-llm-templates-and-agents-8793f30f1837

# NEXT
We can see that GPT-4 is powerful and can translate a natural language question into the right steps in python in order to query a CSV data loaded into a pandas dataframe. 
That's pretty amazing. However the question remains: **Do I need then to dump all the data from my original sources (Databases, ERP Systems, CRM Systems) in order to be searchable by a Smart Search Engine?**

The next Notebook answers this question by implementing a Question->SQL process and get the information from data in a SQL Database, eliminating the need to dump it out.